In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from autograd import jacobian
from numba import jit, float64, int64
import time
import seaborn as sns
import numpy as np
import random
import warnings
warnings.filterwarnings("ignore")


In [2]:
def posi_defnite_check(A):
    """
    check whether the matrix is positive definite
    """
    return np.all(np.linalg.eigvals(A) > 0)

def batch_fun(data, size):
    """
    function for data batch
    """
    n = data.shape[0]
    i = np.arange(n)
    p = data.shape[1]
    
    ## make sure the rows can be divided by the input size
    if n % size !=0:
        print("The number of observations of data cannot be divided by the input batch size.")
        print('%d data dropped.' % (n%size))
        
    sample_size = (n // size) * size
        

    np.random.shuffle(i)

    batch_nums = n//size
    data = data[i][:sample_size].reshape(size,p,batch_nums)
    return(data, batch_nums)

def sghmc(gradU, eps, C, Mmatrix, theta_initial, Cov_hat, epoch_nums, epoch_nums_drop, data, size, seed=663):
    """
    function to apply sghmc 
    """
    
    ## gradU: function(theta, X, y), U gradient

    ## eps: learning rate

    ## C: friction matrix P X P

    ## Mmatrix: Mass matrix P X P

    ## theta_initial: initial value of theta

    ## Cov_hat: estimated covariance matrix of stochastic gradient noise

    ## epoch_nums: number of epochs to perform

    ## epoch_nums_drop: number of epochs to drop

    ## size: minbatch size per iteration

    ## seed: seed
    np.random.seed(seed)
    n = data.shape[0]
    p = theta_initial.shape[0]

    theta_samp = np.zeros((p, epoch_nums))
    theta_samp[:, 0] = theta_initial

    B_hat = 0.5 * eps * Cov_hat

    if not posi_defnite_check(2 * (C - B_hat) * eps):
        print("The noise term has to be positive define.")
        return

    noise_sqrt = np.linalg.cholesky(2 * (C - B_hat) * eps)

    r = (np.linalg.cholesky(np.linalg.inv(Mmatrix))) @ np.random.normal(size=p).reshape(p, -1)
    data_batched = batch_fun(data, size)[0]
    batch_nums = batch_fun(data, size)[1]

    for i in range(epoch_nums - 1):
        r = (np.linalg.cholesky(np.linalg.inv(Mmatrix))) @ np.random.normal(size=p).reshape(p, -1)
        theta = theta_samp[:, i]

        for batch in range(batch_nums):
            theta += (eps * Mmatrix @ r).ravel()
            gradU_batch = gradU(theta, data_batched[:, :, batch], n, size).reshape(p, -1)
            r = r - eps * gradU_batch - eps * C @ Mmatrix @ r + np.random.multivariate_normal(np.zeros(p), noise_sqrt).reshape(p, -1)
           
        theta_samp[:, i + 1] = theta

    return theta_samp[:, epoch_nums_drop:]

In [3]:
import autograd.numpy as np

In [4]:
@jit
def batch_numba(data, size,seed=663):
    """
    function for data batch
    """
    n = data.shape[0]
    p = data.shape[1]
    i = np.arange(n)
    if n % size !=0:
        print("The number of observations of data cannot be divided by the input batch size.")
        print('%d data dropped.' % (n%size))
        
    sample_size = (n // size)*size
        
    np.random.seed(seed)
    np.random.shuffle(i)
    batch_nums = n//size
    data = data[i][:sample_size].reshape(size, p, batch_nums)

    return(data, batch_nums)


In [5]:
@jit
def posi_defnite_check(A):
    """
    check whether the matrix is positive definite - jit
    """
    return np.all(np.linalg.eigvals(A) > 0)

In [6]:
@jit(cache = True)

def sghmc_numba(gradU, eps, C, Mmatrix, theta_initial, Cov_hat, epoch_nums, epoch_nums_drop, data, size,seed=663):
    """
    function to apply sghmc with jit
    """
    
        ## gradU: function(theta, X, y), U gradient
        
        ## eps: learning rate
        
        ## C: friction matrix P X P
        
        ## Mmatrix: Mass matrix P X P
        
        ## theta_initial: initial value of theta
        
        ## Cov_hat: estimated covariance matrix of stochastic gradient noise
        
        ## epoch_nums: number of epochs to perform
        
        ## epoch_nums_drop: number of epochs to drop
        
        ## size: minbatch size per iteration
    
    random.seed(seed)
    n = data.shape[0]
    p = theta_initial.shape[0]

    theta_samp = np.zeros((p, epoch_nums))
    theta_samp[:,0] = theta_initial
    
    B_hat = 0.5*eps*Cov_hat
    
    if not posi_defnite_check(2*(C-B_hat)*eps):
        print("The noise term has to be positive define.")
        return
    
    noise_sqrt = np.linalg.cholesky(2*(C-B_hat)*eps)
    
    r = (np.linalg.cholesky(np.linalg.inv(Mmatrix)))@np.random.normal(size = p).reshape(p, -1)
    data_batched = batch_numba(data, size)[0]
    batch_nums = batch_numba(data, size)[1]
    
    for i in range(epoch_nums-1):
        r = (np.linalg.cholesky(np.linalg.inv(Mmatrix)))@np.random.normal(size = p).reshape(p, -1)
        theta = theta_samp[:,i]
        
        for batch in range(batch_nums):
            theta += (eps * Mmatrix @ r).ravel()
            gradU_batch = gradU(theta, data_batched[:,:, batch], n, size).reshape(p, -1)
            r = r-eps*gradU_batch - eps*C@Mmatrix@r + noise_sqrt@np.random.normal(size = p).reshape(p, -1)
        theta_samp[:,i+1] = theta
    
    return theta_samp[:, epoch_nums_drop:]

    

In [7]:
mu = np.array([-3,3]).reshape(2,1)

def prior1(theta):
    return ((-1/(2*10))*theta.T@theta)

def p_data(theta, x):
    return (np.log(0.5 * np.exp(-0.5*(theta[0]-x)**2) + 0.5* np.exp(-0.5*(theta[1]-x)**2)))

def U(theta, x, n, size):
    return (-prior1(theta) - (n/size)*sum(p_data(theta, x)))




In [8]:
@jit
def prior1_numba(theta):
    return ((-1/(2*10))*theta.T@theta)

In [9]:
@jit
def p_data_numba(theta, x):
    return (np.log(0.5 * np.exp(-0.5*(theta[0]-x)**2) + 0.5* np.exp(-0.5*(theta[1]-x)**2)))

In [10]:
@jit
def U_numba(theta, x, n, size):
    return (-prior1_numba(theta) - (n/size)*sum(p_data_numba(theta, x)))

In [11]:
gradU = jacobian(U, argnum = 0)

gradU_numba = jacobian(U_numba, argnum = 0)

np.random.seed(123)
n = 100
x = np.r_[
    np.random.normal(mu[0], 1, n),
    np.random.normal(mu[1], 1, n)].reshape(-1,1)

theta_0 = np.array([-3, 3])
eps = 0.01
V_hat = np.eye(2)
C = np.eye(2)
epochs = 500
burns = 200
size = 50

In [12]:
%timeit sghmc(gradU, eps, C, np.eye(2), theta_0, V_hat, epochs, burns, x, size)
%timeit sghmc_numba(gradU, eps, C, np.eye(2), theta_0, V_hat, epochs, burns, x, size)

7.61 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.14 s ± 88.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
